In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scripts.data_import import data_import
from scripts.run import run_model

# Initialization

In [2]:
# Data extraction
file = 's5'
path = 'data/' + file + '.xlsx'
data, basis, bess, service, revenue_change = data_import(path)

In [3]:
# Optimization settings
model_optimality_gap = 0.005     # proximity to optimal solution, 0.01 means 1% difference with the optimal solution
model_time_limit = 10         # Maximum time limit in seconds. If 365 days data are available, the model will be solved 365 times, and the total time required = 10 x 365 = 3650s

# Base case simulation

In [4]:
# battery capacity settings (activate to override excel settings)
# bess['cap_power'] = 1
# bess['cap_energy'] = 4

# Scenario analysis (activate to override excel settings)
## To reduce the minimum soc limit
# data['soc_limit']['min'] = 0.1

## To keep 15% regulation up signal
# service['reg_activate'] = 1
# data['activation']['reg'] = 0.15
# data['activation']['reg_up'] = data['activation']['reg'].apply(lambda x: x if x > 0 else 0)
# data['activation']['reg_down'] = data['activation']['reg'].apply(lambda x: -x if x < 0 else 0)

## To keep 15% regulation down signal
service['reg_activate'] = 1
data['activation']['reg'] = -0.15
data['activation']['reg_up'] = data['activation']['reg'].apply(lambda x: x if x > 0 else 0)
data['activation']['reg_down'] = data['activation']['reg'].apply(lambda x: -x if x < 0 else 0)

In [ ]:
# Result file name
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
result_html_name = file + "_" + current_datetime
result_html_path = "results/" + result_html_name

# Run model
packaged_data = [data, basis, bess, service] 
annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_path, model_optimality_gap = model_optimality_gap, model_time_limit = model_time_limit)
[NPV, ROI, IRR, BCR, PP] = financial_metrics


# Sensitivity analysis (multiple energy and power capacity settings)

In [ ]:
power_range = [x * 0.5 for x in range(1, 3)]
energy_range = [x * 0.5 for x in range(1, 3)]

econs_sens_keys = []
econs_sens_values = []

for power_target in power_range:
    for energy_target in energy_range:

        print("Solving: ", power_target, energy_target)

        bess['cap_power'] = power_target
        bess['cap_energy'] = energy_target

        packaged_data = [data, basis, bess, service] 
        annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_path = [], model_optimality_gap = model_optimality_gap, model_time_limit = model_time_limit)
        [NPV, ROI, IRR, BCR, PP] = financial_metrics

        econs_sens_keys.append([power_target, energy_target])
        econs_sens_values.append([annual_revenue] + financial_metrics)

df = pd.DataFrame(index = pd.MultiIndex.from_tuples(econs_sens_keys, names = ['Power', 'Energy']), data = econs_sens_values, columns = ['annual_revenue', 'NPV', 'ROI', 'IRR', 'BCR', 'PP'])
df['annual_revenue_per_kw'] = df['annual_revenue']/df.index.get_level_values(0)/1000
df = df.reset_index()


In [ ]:
# Export result to excel
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_name = "sens_result" + "_" + file + "_" + current_datetime
excel_path = "results/" + excel_name + ".xlsx"
df.to_excel(excel_path)

In [ ]:
# Calculating Energy/Power ratio for the contour plot
df['Ratio'] = df['Energy'] / df['Power']
pivot_df = df.pivot(index="Energy", columns="Power", values="ROI") * 100
pivot_ratio = df.pivot(index="Energy", columns="Power", values="Ratio")

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap with contour lines
fig = go.Figure()

# Add heatmap
fig.add_trace(go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  # High values red, low values blue
    colorbar=dict(title='ROI (%)'),
    zmin=np.min(z),  # Define the range of z for better contrast
    zmax=np.max(z)
))

# Add contour lines for the Energy/Power ratio with text labels
fig.add_trace(go.Contour(
    z=pivot_ratio.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    showscale=False,  # Hide the scale for contour lines
    contours=dict(
        coloring='lines',
        showlabels=True,  # Enable labels on contours
        labelfont=dict(size=12, color='black'),  # Customize the font of the labels
        start=0.5,  # Starting value for contour lines
        end=4,     # Ending value for contour lines
        size=0.5   # Interval between contour lines
    ),
    line=dict(color='black', width=2),
    # ncontours=50  # Increase the number of contour lines

))

# Adding titles and labels, and setting axis ranges to start from 1
fig.update_layout(
    title='Heatmap of ROI with Energy/Power Ratio Contour Lines',
    xaxis=dict(title='Power', range=[0.5, 8]),
    yaxis=dict(title='Energy', range=[0.5, 8])
)

# Display the figure
fig.show()
